In [1]:
import streamlit as st
import pandas as pd
import pydeck as pdk
import hashlib
import glob
import streamlit.components.v1 as components
import plotly.graph_objects as go
import matplotlib.colors as mcolors

In [2]:
# 📂 Ruta donde están los archivos
ruta_archivos = "reportes/*.xlsx"

# 🧾 Lista de todos los archivos que coincidan
archivos = glob.glob(ruta_archivos)

# 📊 Leer y concatenar todos los archivos
df_mrk = pd.concat(
    [pd.read_excel(archivo, sheet_name="Resumen ruta") for archivo in archivos],
    ignore_index=True
)

In [3]:
from datetime import time

# 🕒 1. Detectar columnas con "(hr)"
cols_hr = [col for col in df_mrk.columns if '(hr)' in col]
print("Columnas a convertir:", cols_hr)

# 🧭 2. Normalizar y convertir
for col in cols_hr:
    # Si algún valor es datetime.time, conviértelo a string "HH:MM:SS"
    df_mrk[col] = df_mrk[col].apply(
        lambda x: x.strftime('%H:%M:%S') if isinstance(x, time) else x
    )
    # Luego convertir a timedelta
    df_mrk[col] = pd.to_timedelta(df_mrk[col], errors='coerce')

# 📊 Verificar
df_mrk.info()

Columnas a convertir: ['Horario de sincronización (hr)', 'Inicio de día (hr)', 'Primer cliente (hr)', 'Tiempo cliente-transmisión (hr)', 'Tiempo de mercado (hr)', 'Tiempo de servicio (hr)', 'Traslado clientes(hr)', 'Transmisión de pedido (hr)', 'Tiempo de jornada (hr)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111135 entries, 0 to 111134
Data columns (total 32 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   Zona                             111135 non-null  object         
 1   Región                           111135 non-null  object         
 2   Gerencia                         111135 non-null  object         
 3   Jefatura                         111135 non-null  object         
 4   Ruta                             111135 non-null  object         
 5   Fecha inicio                     111135 non-null  datetime64[ns] 
 6   Fecha fin                        111135 non-n

In [4]:
df_mrk.head()

,Zona,Región,Gerencia,Jefatura,Ruta,Fecha inicio,Fecha fin,Geoefectividad,Efectividad omnicanal,Geoeficiencia,...,Tiempo de mercado (hr),Tiempo de servicio (%),Tiempo de servicio (hr),Traslado clientes (%),Traslado clientes(hr),Transmisión de pedido (%),Transmisión de pedido (hr),Tiempo de jornada (%),Tiempo de jornada (hr),Usabilidad
0,BAJIO,CELAYA,AC01,AC0,ACK001,2025-10-27,2025-10-27,0.878,0.951,0.927,...,0 days 12:00:57,0.292,0 days 03:23:26,0.975,0 days 07:38:33,0,0 days 19:13:32,0,0 days 13:29:58,0.166
1,BAJIO,CELAYA,AC01,AC2,AC0067,2025-10-27,2025-10-27,0.432,0.795,0.818,...,0 days 10:42:21,0.159,0 days 02:14:14,0.930,0 days 06:34:02,0,0 days 19:34:02,0,0 days 13:04:41,0.333
2,BAJIO,CELAYA,AC01,AC2,AC0075,2025-10-27,2025-10-27,0.510,0.795,0.959,...,0 days 08:32:06,0.040,0 days 01:40:58,0.877,0 days 04:53:13,1,0 days 18:19:10,0,0 days 12:20:31,0.116
3,BAJIO,CELAYA,AC01,AC2,AC0076,2025-10-27,2025-10-27,0.604,0.679,0.981,...,0 days 09:32:02,0.132,0 days 02:38:28,0.962,0 days 04:29:46,1,0 days 18:53:08,0,0 days 12:48:53,0.078
4,BAJIO,CELAYA,AC01,AC2,AC0077,2025-10-27,2025-10-27,0.820,0.959,0.880,...,0 days 09:06:33,0.140,0 days 02:45:18,0.900,0 days 05:28:43,1,0 days 18:45:22,0,0 days 12:40:03,0.348


In [5]:
df_catalogo = pd.read_csv('catalogo.csv', encoding='utf-8')
df_catalogo = df_catalogo[['Ruta','Tipo','Descripción Tipo','TPV','Región Comercial_Act 2026']]

df_mrk_full = df_mrk.merge(how='left', right=df_catalogo, left_on='Ruta', right_on='Ruta')
df_mrk_full.head()

,Zona,Región,Gerencia,Jefatura,Ruta,Fecha inicio,Fecha fin,Geoefectividad,Efectividad omnicanal,Geoeficiencia,...,Traslado clientes(hr),Transmisión de pedido (%),Transmisión de pedido (hr),Tiempo de jornada (%),Tiempo de jornada (hr),Usabilidad,Tipo,Descripción Tipo,TPV,Región Comercial_Act 2026
0,BAJIO,CELAYA,AC01,AC0,ACK001,2025-10-27,2025-10-27,0.878,0.951,0.927,...,0 days 07:38:33,0,0 days 19:13:32,0,0 days 13:29:58,0.166,PRTDC,Preventa Especializada TDC,ZPV,Celaya
1,BAJIO,CELAYA,AC01,AC2,AC0067,2025-10-27,2025-10-27,0.432,0.795,0.818,...,0 days 06:34:02,0,0 days 19:34:02,0,0 days 13:04:41,0.333,PR,Preventa Comercial,ZPV,Celaya
2,BAJIO,CELAYA,AC01,AC2,AC0075,2025-10-27,2025-10-27,0.510,0.795,0.959,...,0 days 04:53:13,1,0 days 18:19:10,0,0 days 12:20:31,0.116,PR,Preventa Comercial,ZPV,Celaya
3,BAJIO,CELAYA,AC01,AC2,AC0076,2025-10-27,2025-10-27,0.604,0.679,0.981,...,0 days 04:29:46,1,0 days 18:53:08,0,0 days 12:48:53,0.078,PR,Preventa Comercial,ZPV,Celaya
4,BAJIO,CELAYA,AC01,AC2,AC0077,2025-10-27,2025-10-27,0.820,0.959,0.880,...,0 days 05:28:43,1,0 days 18:45:22,0,0 days 12:40:03,0.348,PR,Preventa Comercial,ZPV,Celaya


In [6]:
df_mrk_cp = df_mrk_full[df_mrk_full['Zona'] == 'CENTRO-PACIFICO']
df_mrk_cp = df_mrk_cp[df_mrk_cp['Tipo'].isin(['PR', 'PCOP', 'FC', 'MA', 'PRTDC'])]

df_mrk_cp['Región'] = df_mrk_cp['Región'].where(
~df_mrk_cp['Tipo'].isin(['MA', 'PRTDC'])
).fillna(df_mrk_cp['Tipo'].replace({'MA': 'MAYORISTAS', 'PRTDC': 'MODERNO'}))
df_mrk_cp['Semana'] = df_mrk_cp['Fecha inicio'].dt.isocalendar().week
df_mrk_cp.rename(columns={'Región Comercial_Act 2026': 'Nombre UO'}, inplace=True)


#df_mrk_cp_pivot = df_mrk_cp.pivot_table(index=['Semana','Fecha inicio'],columns='Región',values=['Geoeficiencia'], aggfunc='mean')

df_mrk_cp
df_mrk_cp.to_parquet('bdd_mrk_cp.parquet')